In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15 
# Title: CCAMLR Subareas and CEMP Sites Map
# R version: 4.3.3
# This script generates a geographic map of selected CCAMLR management subareas (ASD) and monitoring sites (CEMP),
# using bathymetry data as a background. Data is clipped, masked, and reprojected.
# Sources: github.com/ccamlr/data & gis.ccamlr.org
# ==============================================================================

# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================

system("conda install -y r-sf=1.0_20 r-terra=1.8_42")
library(terra)     
library(dplyr)
library(sf)        
library(jsonlite) 

# ==============================================================================
# 1. PARAMETERS AND INPUT LOADING ----
# ==============================================================================

options(timeout=3600)
json_data     <- fromJSON("galaxy_inputs/galaxy_inputs.json")
subarea       <- json_data$subarea
area_list     <- trimws(unlist(strsplit(subarea, ",")))
m             <- json_data$multiplier
Zone_ASD      <- json_data$ZoneXASD
Label_ASD     <- json_data$LabelXASD
Zone_CEMP     <- json_data$ZoneXCEMP
bbox          <- json_data$bbox
png           <- json_data$png




# ==============================================================================
# 2. LOAD DATA ----
# ==============================================================================

# 2.1 DOWNLOAD AND LOAD ASD AND CEMP ----
output_dir <- "asd"
output_dir_cemp <- "cemp"

dir.create(output_dir, showWarnings = FALSE)
dir.create(output_dir_cemp, showWarnings = FALSE)

urls_asd <- c(
  paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.", c("shp", "shx", "dbf", "prj")),
  paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.", c("shp", "shx", "dbf", "prj"))
)
urls_cemp <- paste0("https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/cemp/cemp-shapefile-EPSG4326.", c("shp", "shx", "dbf", "prj"))

lapply(seq_along(urls_asd), function(i) download.file(urls_asd[i], file.path(output_dir, basename(urls_asd[i])), mode = "wb"))
lapply(seq_along(urls_cemp), function(i) download.file(urls_cemp[i], file.path(output_dir_cemp, basename(urls_cemp[i])), mode = "wb"))

ASDs <- st_read(file.path(output_dir, "asd-shapefile-EPSG4326.shp"))
CEMP <- st_read(file.path(output_dir_cemp, "cemp-shapefile-EPSG4326.shp"))

# Reproject to the correct CRS --- 
ASDs <- st_transform(ASDs, crs = 6932)
CEMP <- st_transform(CEMP, st_crs(ASDs))

# 2.2 DOWNLOAD AND LOAD BATHYMETRY ----
bathy_file <- "GEBCO2024_5000.tif"
download.file("https://gis.ccamlr.org/geoserver/www/GEBCO2024_5000.tif", bathy_file, mode = "wb")
Bathy <- rast(bathy_file)

# 2.3 MASK AND CROP BATHYMETRY ----
subareas_selected <- ASDs %>% filter(GAR_Short_ %in% area_list )

if(bbox){
  subareas_bbox <- vect(st_as_sfc(st_bbox(subareas_selected)))
  Bathy_mask <- mask(crop(Bathy, subareas_bbox), subareas_bbox)
} else{
    Bathy_mask <- mask(crop(Bathy, vect(subareas_selected)),vect(subareas_selected))
}

Bathy_extent <- ext(Bathy_mask)
w= abs(Bathy_extent[2] - Bathy_extent[1])/10000
h=abs(Bathy_extent[4] - Bathy_extent[3])/10000 

# ==============================================================================
# 3. MAP PLOTTING ----
# ==============================================================================

if (png) {
  png(filename = "outputs/Fig1.png", width = w*m, height = h*m)
    
} else {
  pdf("outputs/Fig1.pdf")
}

Depth_cuts <- c(-8200 ,-7000 ,-6000 ,-5000, -4000, -3000, -1800, -1400, -1000,  -600,  -400 , -200  ,   0 ,   50  , 250   ,500)
Depth_cols <- c(
  "#D6EAF8", "#AED6F1", "#85C1E9", "#5DADE2", "#3498DB",
  "#5DADE2", "#85C1E9", "#A9CCE3", "#D4E6F1", "#EBF5FB",
  "#F4F6F7", "#F8F9F9", "#FDFEFE", "#F2F3F4", "#EAEDED"
)

# Plot bathymetry
plot(Bathy_mask,breaks = Depth_cuts, col = Depth_cols, legend = FALSE, axes = FALSE, box = FALSE,mar=c(0,0,0,0))

# Add ASD boundaries in red ---
if (Zone_ASD) {
    plot(st_geometry(subareas_selected), add = TRUE, lwd = 0.75*m, border = 'red')
}

if (Label_ASD) {
    zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
    zones <- ASDs %>% filter(GAR_Short_ %in% area_list)
    centroids <- st_centroid(st_geometry(zones))
    text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 1*m) 
}

# Add CEMP sites as orange points ---
if (Zone_CEMP) {
    plot(st_geometry(CEMP), add = TRUE, pch = 21, bg = 'orange', col = 'black', cex = 2.5)
}

# Save the plot ---
dev.off()




